In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, 
    T5TokenizerFast, T5ForConditionalGeneration, 
    AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, 
    AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
)

from datasets import load_metric, Dataset

import wandb
import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(6, 64)

In [3]:
torch.cuda.is_available()

True

# Paths and Names

In [4]:
### paths and names

PROJECT_NAME = 'news-topic-keyphrase-generation-model-dev'
RUN_ID = 'v3_run_3'

DATA_PATH = 'data/model_dev/model_dev_v3.pickle'

MODEL_CHECKPOINT = 'paust/pko-t5-base'
model_name = re.sub(r'[/-]', r'_', MODEL_CHECKPOINT).lower()

METRIC_NAME = 'rouge'

NOTEBOOK_NAME = './train_seq2seq_plm.ipynb'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, '.log')

run_name = f'{model_name}_{RUN_ID}'
output_dir = os.path.join(SAVE_PATH, run_name)

print(run_name)
print(output_dir)

!mkdir -p {SAVE_PATH}

paust_pko_t5_base_v3_run_3
./.log/paust_pko_t5_base_v3_run_3


In [5]:
os.environ['WANDB_PROJECT'] = PROJECT_NAME
os.environ['WANDB_NOTEBOOK_NAME'] = NOTEBOOK_NAME
os.environ['WANDB_LOG_MODEL'] = 'true'
os.environ['WANDB_WATCH'] = 'all'

wandb.login()

wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Training Args

In [6]:
report_to="wandb"

num_train_epochs = 15
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1

optim = 'adamw_torch' # 'adamw_torch' or 'adamw_hf'

learning_rate = 3e-6 * (per_device_train_batch_size * NGPU) / 8
weight_decay = 0.01
adam_epsilon = 1e-8

lr_scheduler_type = 'cosine' # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model = 'eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = int(500 / NGPU)

predict_with_generate=True
generation_max_length=64
# generation_num_beams=5

fp16 = False

# Model & Tokenizer & Metric

In [7]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = load_metric(METRIC_NAME)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Functions

In [9]:
prefix = "generate keyphrases: "

max_input_length = 1024
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Inputs and Labels

In [11]:
data_df = pd.read_pickle(DATA_PATH)

In [12]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2, seed=100)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [13]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

Map (num_proc=64):   0%|          | 0/9346 [00:00<?, ? examples/s]

Map (num_proc=64):   0%|          | 0/2337 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9346
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2337
})


In [14]:
tokenizer.decode(train_dataset['input_ids'][0])

'generate keyphrases: "로페테기가 좋아하는 황희찬, 리즈전 선발 가능성 있어" 복귀전에서 득점을 한 황희찬은 선발 출전을 대기하고 있다.울버햄튼은 19일 오전 0시(한국시간) 영국 울버햄튼에 위치한 몰리뉴 스타디움에서 리즈 유나이티드와 2022-23시즌 잉글리시 프리미어리그(EPL) 28라운드를 치른다. 울버햄튼은 13위, 리즈는 19위에 위치 중이다.영국 \'익스프레스 앤 스타\'는 울버햄튼, 리즈 경기를 프리뷰하면서 "홈 팬들은 시작부터 공격적이고 강렬한 경기를 원한다. 훌렌 로페테기 감독은 때로는 보수적인 방식으로 접근을 했다. 홈 팬들 앞에서는 달라야 한다. 공격진 변화가 예상되는 황희찬이 선발로 나설 수 있다"고 전했다.황희찬은 브루노 라즈 감독 아래에선 벤치 신세였다. 로페테기 감독이 온 후엔 달랐다. 득점 수는 적어도 기동력과 저돌적인 황희찬을 선호했다. 경기력으로 응답했다. 2022 국제축구연맹(FIFA) 카타르 월드컵에서 활약으로 자신감까지 올랐다. 좋은 활약을 이어가던 황희찬은 리버풀전에서 부상을 당해 한동안 빠졌다.재활 기간을 거친 황희찬은 지난 뉴캐슬 유나이티드전에 복귀를 했는데 골을 넣었다. 0-1로 뒤지던 후반 24분 들어와 후반 25분 집중력 있는 모습으로 뉴캐슬 골망을 흔들었다. 경기는 울버햄튼의 1-2 패배로 끝이 났지만 황희찬에겐 고무적인 날이었다. 이날 골로 황희찬은 리그 1호 득점에 성공했고 EPL에선 무려 13개월 만에 골 맛을 봤다. 부상 불운을 골로 보답을 받는 날이었다.\'익스프레스 앤 스타\'는 "로페테기 감독은 황희찬을 정말 좋아한다. 근면한 모습과 라울 히메네스를 대체할 수 있는 능력이 있어 리즈전에 선발로 택할 수 있다. 지난 주말 햄스트링 부상에서 돌아온 황희찬은 뉴캐슬을 상대로 골을 넣었다. 황희찬이 선발로 나오면 레프트백 라얀 아이트-누리와 같이 가능성이 높다"고 구체적인 전망을 내놓았다.울버햄튼 순위는 13위지만 강등권인 18위 본머스와 승점 3점차밖에 안 난다. 리즈전 필승이 요구되는

In [15]:
tokenizer.decode(train_dataset['labels'][0])

'로페테기; 황희찬; 선발 출전; 울버햄튼; 리즈전; EPL; 득점; 복귀전; 한동안; 부상</s>'

# Train

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,

    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
    # generation_num_beams=generation_num_beams,

    fp16=fp16,
)

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
    
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()
wandb.finish()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(output_dir)
for ckpt in ckpts:
    ckpt = os.path.join(output_dir, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

# Generate

In [ ]:
preds = trainer.predict(eval_dataset)

In [ ]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')